In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [59]:
from IPython.display import HTML, Markdown

In [13]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7

In [58]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [15]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [16]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [25]:
##########################################
# --- Agente 1: Agente de Busca --- #
##########################################

def agente_buscador(topico, data_de_hoje):
  buscador = Agent(
      name="agente_buscador",
      model="gemini-2.0-flash",
      description="Agente que busca notícias sobre o tópico indicado",
      tools=[google_search],
      instruction="""Agente de Busca (google_search)
Objetivo: Reunir todas as informações essenciais e complementares necessárias para que os outros agentes possam desenvolver o conteúdo de forma precisa e completa.

Instruções:

Levantamento de Requisitos: Leia atentamente o texto fornecido e identifique todas as características, funcionalidades, comportamentos e requisitos de dados do chatbot.
Pesquisa de Contexto:
Para o "Comportamento": Busque exemplos de gírias locais de pesca no Brasil que possam ser usadas com cuidado.
Para as "Funcionalidades" (Exemplos): Verifique se há padrões ou informações adicionais sobre os marcadores de posição [Nome do Rio/Mar/Lagoa], [Nome do Peixe], [Tipo de Estrutura], [Ponto geográfico específico], [Hora Início], [Hora Fim], [Cheia/Vazante], [Condição Climática], [Direção], [Velocidade], [Temperatura], [Tipo de Isca], [Tipo de Isca Artificial], [Cor], [Ex: vara de 10-20 libras, linha 0.30mm]. Embora o texto forneça exemplos, o objetivo é garantir que não haja lacunas conceituais.
Para "Dados": Pesquise brevemente sobre a natureza e o acesso a "dados de localização de peixes (relatos de pescadores, institutos de pesquisa, padrões migratórios)", "dados meteorológicos em tempo real e tabelas de marés", e "informações sobre espécies de peixes" para ter uma compreensão básica de como essas informações são obtidas.
Análise do Exemplo de Interação: Discuta o exemplo de interação para entender o fluxo e a aplicação das funcionalidades e do comportamento.
Consolidação: Organize as informações levantadas em um formato claro e acessível para o Agente de Planejamento e o Agente Redator.
"""

  )

  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
  lancamentos= call_agent(buscador, entrada_do_agente_buscador)
  return lancamentos

In [37]:
################################################
# --- Agente 2: Agente de Planejamento --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Agente de Planejamento
Objetivo: Estruturar o conteúdo de forma lógica e coerente, definindo a sequência de tópicos e garantindo que todas as informações importantes sejam abordadas.

Instruções:

Revisão do Conteúdo Bruto: Analise o material coletado pelo Agente de Busca.
Definição da Estrutura Principal:
O chatbot adota um tom adulto, amigável e experiente, como um pescador veterano, utilizando poucas gírias locais de forma cuidadosa e apropriada.
Suas respostas são diretas e claras, evitando jargões técnicos sempre que possível, e utilizam negrito para destacar informações importantes e raramente emojis para uma conversa mais visual e leve.
O sistema é capaz de compreender as perguntas dos pescadores, mesmo que não sejam exatas, e interpretar a localização do usuário para sugerir os melhores pontos de pesca nas proximidades.
Se aprofunde nos temas: (Localização de Peixes, Melhores Horários, Condições Climáticas,Estação do ano, Iscas Recomendadas, Equipamentos Necessários e Dicas) use pelo menos 3 linhas de texto em cada para que não fique muito vago.
As interações iniciam com uma saudação amigável, como: Olá, companheiro de pesca! Como posso ajudar hoje?, e terminam com uma despedida encorajadora, como: Boa pescaria e volte sempre! Se precisar, é só chamar!.
Destaques e Ênfase: Identifique quais informações devem ser destacadas (por exemplo, usando negrito) para maior clareza e impacto.
Entrega: Apresente um plano detalhado para o Agente Redator, incluindo a estrutura do documento, os pontos-chave a serem abordados em cada seção e as informações a serem enfatizadas.
        """,
        description="Agente que estrutura o conteúdo.",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [48]:
######################################
# --- Agente 3: Agente Redator --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Agente Redator
Objetivo: Escrever o conteúdo de forma clara, concisa e envolvente, seguindo as diretrizes de comportamento do chatbot (tom amigável e experiente, linguagem direta, uso de negrito e emojis, quando apropriado).

Instruções:

Consulta ao Plano: Utilize o plano de estrutura fornecido pelo Agente de Planejamento e as informações detalhadas pelo Agente de Busca.
Elaboração da Introdução: Escreva uma introdução cativante que apresente o chatbot e seu propósito para os pescadores brasileiros.
Desenvolvimento das Seções:
Aprofunde-se nos temas: (Localização de Peixes, Melhores Horários, Condições Climáticas, Estação do ano, Iscas Recomendadas, Equipamentos Necessários e Dicas) para que não falte informação.
Comportamento: Suas respostas são diretas e claras, evitando jargões técnicos sempre que possível, e utilizam negrito para destacar informações importantes e emojis para uma conversa mais visual e leve.
Funcionalidades: O sistema é capaz de compreender as perguntas dos pescadores, mesmo que não sejam exatas, e interpretar a localização do usuário para sugerir os melhores pontos de pesca nas proximidades.
Exemplo de Interação: As interações iniciam com uma saudação amigável, como: Olá, companheiro de pesca! Como posso ajudar hoje?, e terminam com uma despedida encorajadora, como: Boa pescaria e volte sempre! Se precisar, é só chamar!.
Aplicação do Comportamento: O chatbot adota um tom adulto, amigável e experiente, como um pescador veterano, utilizando poucas gírias locais de forma cuidadosa e apropriada.
Coerência e Clareza: Mantenha a fluidez entre as seções e evite ao maximo jargões desnecessários na descrição do chatbot.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [55]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        tools=[google_search],
        instruction="""
            Agente Revisor
Objetivo: Garantir a qualidade final do texto, verificando a gramática, ortografia, pontuação, clareza, coerência e a aderência às instruções originais e ao comportamento desejado do chatbot.

Instruções:

Revisão Abrangente: Leia o texto produzido pelo Agente Redator.
Verificação Gramatical e Ortográfica: Corrija quaisquer erros de gramática, ortografia e pontuação.
Use o (google_search) para verificar se a informação ainda é válida.
Clareza e Coerência:
Verifique se a linguagem é adulta (entre 20 a 50 anos), clara e de fácil compreensão.
Assegure-se de que o texto flui de forma lógica e que as informações estão bem conectadas.
Confirme se não há redundâncias ou informações conflitantes.
Verifique se as informações de linha de pesca estáo entre 1.00mm e 0.50mm, se estiver abaixo de 0.50mm set para 0.50mm (exemplo de erra: Equipamento: Vara média, linha 0,30mm e anzol médio. , correção: Equipamento: Vara média, linha 0,50mm e anzol médio.)
Aderência ao Comportamento do Chatbot:
Avalie se o tom amigável e experiente foi mantido consistentemente na descrição do chatbot.
Verifique o uso apropriado de negrito e a ausência de jargões técnicos na descrição do chatbot.
Exatidão das Informações: Compare o texto final com o material original e as informações do Agente de Busca e Planejamento para garantir a precisão de todas as características e funcionalidades descritas.
Formatação: Revise a formatação (negrito, espaçamento, etc.) para garantir que o texto seja visualmente atraente e fácil de ler.
Sintonia com os Exemplos: Confirme se os exemplos de funcionalidades e o exemplo de interação final estão bem integrados e são representativos.
            """,
        description="Agente reviso"
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Ligando o motor 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("🐟 Olá Pescador! Como posso ajudar??🐟 " )

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Por favor, insira um tópico válido.")
else:
  print(f"🎣 Pesquisando sobre {topico}... 🎣")

  lancamentos_buscados = agente_buscador(topico, data_de_hoje)
  print("_____Resultado agente 1_____")
  display(to_markdown(lancamentos_buscados))
  print("____________________________")

  lancamentos_planejamento = agente_planejador(topico, lancamentos_buscados)
  print("_____Resultado agente 2_____")
  display(to_markdown(lancamentos_planejamento))
  print("____________________________")

  lancamentos_rascunho = agente_redator(topico, lancamentos_planejamento)
  print("_____Resultado agente 3_____")
  display(to_markdown(lancamentos_rascunho))
  print("____________________________")

  resultado_final = agente_revisor(topico, lancamentos_rascunho)
  print("_____Resultado agente 4_____")
  display(to_markdown(resultado_final))
  print("____________________________")


🚀 Ligando o motor 🚀
